
# ClickBus — Entrega Completa (v5)
**Segmentação de clientes • Próxima compra (7/30d) • Dias até próxima • Próximo trecho • Explainability • Artefatos**
  
Este notebook foi desenhado para **Google Colab** e salva artefatos diretamente no **Google Drive** em `/content/drive/MyDrive/clickbuss`.


## 0. Setup & caminhos

In [ ]:

# Instala pacotes necessários (Colab)
!pip install -q lightgbm xgboost catboost holidays optuna shap lifelines python-pptx category_encoders

import os, warnings, numpy as np, pandas as pd, joblib, matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# Saídas
OUTPUT_DIR = '/mnt/data'  # para baixar aqui pela conversa
DRIVE_OUT = '/content/drive/MyDrive/clickbuss'  # para salvar no seu Drive
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('OUTPUT_DIR:', OUTPUT_DIR)
print('DRIVE_OUT:', DRIVE_OUT)


OUTPUT_DIR: /mnt/data
DRIVE_OUT: /content/drive/MyDrive/clickbuss


## 1. Montar Drive & carregar dados

In [ ]:

# Montar Google Drive
try:
  from google.colab import drive
  drive.mount('/content/drive')
  DRIVE_MOUNTED = True
  os.makedirs(DRIVE_OUT, exist_ok=True)
except Exception as e:
  print('Drive não disponível (rodando local?).', e)
  DRIVE_MOUNTED = False

# Caminhos do CSV
drive_csv = '/content/drive/MyDrive/df_t.csv'
local_csv = 'df_t.csv'

# Carregamento
if DRIVE_MOUNTED and os.path.exists(drive_csv):
    df = pd.read_csv(drive_csv, low_memory=False)
elif os.path.exists(local_csv):
    df = pd.read_csv(local_csv, low_memory=False)
else:
    raise FileNotFoundError('df_t.csv não encontrado em /content/drive/MyDrive/ nem no diretório atual.')
print('Shape:', df.shape)
display(df.head(3))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape: (1741344, 12)


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,gmv_success,total_tickets_quantity_success
0,bc02d5245bec63b30ff1102fa273fc03f58bc9cc3f674e...,a7218ff4ee7d37d48d2b4391b955627cb089870b934912...,2018-12-26,15:33:35,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,50e9a8665b62c8d68bccc77c7c92431a1aa26ccbd38ed4...,0,0,8527a891e224136950ff32ca212b45bc93f69fbb801c3b...,1,89.09,1
1,5432f12612dd5d749b3be880e779989cf63b5efa4bcc4e...,37228485e0dc83d84d1bcd1bef3dc632301bf6cb22c8b5...,2018-12-05,15:07:57,10e4e7caf8b078429bb1c80b1a10118ac6f963eff098fd...,e6d41d208672a4e50b86d959f4a6254975e6fb9b088116...,0,0,36ebe205bcdfc499a25e6923f4450fa8d48196ceb4fa0c...,1,155.97,1
2,fb3caed9b2f1b6016d45ccddb19095476e61a2c85faa8e...,3467ec081e2421e72c96e7203b929d21927fd00b6b5f28...,2018-12-21,18:41:54,7688b6ef52555962d008fff894223582c484517cea7da4...,8c1f1046219ddd216a023f792356ddf127fce372a72ec9...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,121.99,1


## 2. Feature Engineering


In [ ]:
import numpy as np
import pandas as pd
import holidays

# Contar número de compras por cliente
compras_por_cliente = df.groupby('fk_contact')['nk_ota_localizer_id' if 'nk_ota_localizer_id' in df.columns else 'nk_order_id'].nunique().reset_index()
compras_por_cliente = compras_por_cliente.rename(columns={compras_por_cliente.columns[1]:'num_compras'})

# Ordenar por clientes que mais compraram
compras_por_cliente = compras_por_cliente.sort_values('num_compras', ascending=False)

# Selecionar top 50k
top_clientes = compras_por_cliente.head(50000)['fk_contact'].tolist()

# Filtrar a base original
df = df[df['fk_contact'].isin(top_clientes)].copy()

print("Base reduzida para:", df.shape, "linhas e", df['fk_contact'].nunique(), "clientes (top compradores)")

# --- Padronização de colunas ---
df.columns = df.columns.str.strip()

# Identificador de venda
order_id_col = 'nk_ota_localizer_id' if 'nk_ota_localizer_id' in df.columns else (
    'nk_order_id' if 'nk_order_id' in df.columns else None
)
if order_id_col is None:
    raise KeyError('Não encontrei nk_ota_localizer_id nem nk_order_id.')

# --- Datas ---
df['date_purchase'] = pd.to_datetime(df['date_purchase'], errors='coerce')
df['time_purchase'] = df['time_purchase'].fillna('00:00:00').astype(str)
df['purchase_datetime'] = pd.to_datetime(
    df['date_purchase'].dt.strftime('%Y-%m-%d') + ' ' + df['time_purchase'],
    errors='coerce'
)

# --- Deduplicação ---
df = df.sort_values(['fk_contact','purchase_datetime']).drop_duplicates(
    subset=[order_id_col,'fk_contact','purchase_datetime'], keep='first'
).reset_index(drop=True)

# --- Rótulos (próxima compra) ---
df['next_purchase_dt'] = df.groupby('fk_contact')['purchase_datetime'].shift(-1)
df['days_to_next'] = (df['next_purchase_dt'] - df['purchase_datetime']).dt.days
df['label_7d']  = (df['days_to_next'] <= 7).astype(int)
df['label_30d'] = (df['days_to_next'] <= 30).astype(int)

# =========================
# Sinais comportamentais sem vazamento
# =========================
df['purchase_number'] = df.groupby('fk_contact').cumcount() + 1

# métricas cumulativas (usando shift para não vazar a compra atual)
df['gmv_shift'] = df.groupby('fk_contact')['gmv_success'].shift(1).fillna(0)
df['tickets_shift'] = df.groupby('fk_contact')['total_tickets_quantity_success'].shift(1).fillna(0)

df['cum_gmv_before']     = df.groupby('fk_contact')['gmv_shift'].cumsum()
df['cum_tickets_before'] = df.groupby('fk_contact')['tickets_shift'].cumsum()
df['avg_ticket_so_far']  = np.where(df['purchase_number']>1,
                                    df['cum_gmv_before']/(df['purchase_number']-1),
                                    0.0)

# intervalos entre compras
df['days_since_prev'] = df.groupby('fk_contact')['purchase_datetime'].diff().dt.days
df['days_since_prev'] = df['days_since_prev'].fillna(df['days_since_prev'].median() if df['days_since_prev'].notna().any() else 0)

# média acumulada (eficiente)
df['avg_days_between'] = (
    df.groupby('fk_contact')['days_since_prev']
      .transform(lambda x: x.cumsum() / (np.arange(len(x)) + 1))
)

# variabilidade de intervalo
# std acumulado aproximado via rolling window crescente (sem incluir a compra atual)
df['interval_std'] = (
    df.groupby('fk_contact')['days_since_prev']
      .transform(lambda x: x.expanding().std(ddof=0))
).fillna(0)

# Coeficiente de variação aproximado
eps = 1e-6
df['interval_cv'] = (df['interval_std'] / (df['avg_days_between'] + eps)).fillna(0)

# preço por bilhete (unit_price) e rolling média anterior (sem vazamento)
df['unit_price'] = (df['gmv_success'] / df['total_tickets_quantity_success'].replace(0, np.nan)).fillna(0)
df['unit_price_rollmean'] = df.groupby('fk_contact')['unit_price'].shift(1).rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).fillna(0)

# --- Últimos categóricos observados
for c in ['place_origin_departure','place_destination_departure',
          'place_origin_return','place_destination_return',
          'fk_departure_ota_bus_company']:
    if c in df.columns:
        df[c+'_last'] = df.groupby('fk_contact')[c].shift(1).fillna('unknown')

# --- Rota ---
df['route'] = df['place_origin_departure'].astype(str) + '->' + df['place_destination_departure'].astype(str)

# Diversidade de rota antes (cumulativa, sem vazamento)
df['route_id_global'] = df['route'].factorize()[0]
# número de rotas únicas vistas ANTES da compra atual
df['unique_routes_before'] = (
    df.groupby('fk_contact')['route_id_global']
      .transform(lambda x: x.expanding().apply(lambda s: len(set(s[:-1])), raw=False))
      .fillna(0)
      .astype(int)
)

# mesma rota da última compra?
df['same_route_as_last'] = (df['route'] == df.groupby('fk_contact')['route'].shift(1)).astype(int).fillna(0)

# =========================
# Temporais, sazonalidade e feriados (1 & 2)
# =========================
df['purchase_hour']     = df['purchase_datetime'].dt.hour.fillna(0).astype(int)
df['purchase_weekday']  = df['purchase_datetime'].dt.weekday.fillna(0).astype(int)  # 0=Mon
df['purchase_month']    = df['purchase_datetime'].dt.month.fillna(0).astype(int)
df['is_weekend']        = df['purchase_weekday'].isin([5,6]).astype(int)

# Sazonalidade simples: alta temporada (jan, jul, dez)
df['is_high_season'] = df['purchase_month'].isin([1,7,12]).astype(int)

# Codificação cíclica (melhora captura de sazonalidade)
df['sin_month'] = np.sin(2*np.pi*df['purchase_month']/12.0)
df['cos_month'] = np.cos(2*np.pi*df['purchase_month']/12.0)
df['sin_wday']  = np.sin(2*np.pi*df['purchase_weekday']/7.0)
df['cos_wday']  = np.cos(2*np.pi*df['purchase_weekday']/7.0)
df['sin_hour']  = np.sin(2*np.pi*(df['purchase_hour']+1e-3)/24.0)
df['cos_hour']  = np.cos(2*np.pi*(df['purchase_hour']+1e-3)/24.0)

# feriados BR
br = holidays.Brazil(years=list(range(2019, 2024)))
holiday_dates = sorted(list(br.keys()))

# é feriado no dia
df['is_holiday'] = df['purchase_datetime'].dt.date.apply(lambda d: 1 if d in br else 0)

# janela do feriado ±7 dias
def in_holiday_window(date, window=7):
    # early exit
    if not holiday_dates:
        return 0
    # busca binária para achar vizinhança mais próxima
    import bisect
    i = bisect.bisect_left(holiday_dates, date)
    neigh = []
    if i < len(holiday_dates): neigh.append(holiday_dates[i])
    if i > 0: neigh.append(holiday_dates[i-1])
    for h in neigh:
        if abs((date - h).days) <= window:
            return 1
    return 0

df['is_holiday_window'] = df['purchase_datetime'].dt.date.apply(lambda d: in_holiday_window(d, window=7))

# distância em dias até o feriado mais próximo (sinal contínuo)
def days_to_nearest_holiday(date):
    import bisect
    i = bisect.bisect_left(holiday_dates, date)
    deltas = []
    if i < len(holiday_dates): deltas.append(abs((holiday_dates[i]-date).days))
    if i > 0: deltas.append(abs((holiday_dates[i-1]-date).days))
    return min(deltas) if deltas else 999

df['days_to_holiday'] = df['purchase_datetime'].dt.date.apply(days_to_nearest_holiday).astype(int)

# --- Agregações por rota ---
route_agg = df.groupby('route').agg(
    route_count=(order_id_col,'nunique'),
    route_avg_gmv=('gmv_success','mean')
).reset_index()
df = df.merge(route_agg, on='route', how='left')

# --- Frequency encodings globais ---
for c in ['place_origin_departure','place_destination_departure',
          'fk_departure_ota_bus_company','route']:
    if c in df.columns:
        vc = df[c].value_counts(normalize=True).to_dict()
        df[c+'_fe'] = df[c].map(vc).fillna(0)

# --- Agregados de cliente (RFM) ---
cust = df.groupby('fk_contact').agg(
    last_purchase=('purchase_datetime','max'),
    first_purchase=('purchase_datetime','min'),
    frequency=(order_id_col,'nunique'),
    monetary=('gmv_success','sum')
).reset_index()
cust['recency_days'] = (df['purchase_datetime'].max() - cust['last_purchase']).dt.days
cust['avg_ticket']    = cust['monetary'] / cust['frequency']
cust['days_active']   = (cust['last_purchase'] - cust['first_purchase']).dt.days.fillna(0)

df = df.merge(cust[['fk_contact','recency_days','frequency',
                    'monetary','avg_ticket','days_active']],
              on='fk_contact', how='left')

# --- Recência em faixas (dummies) ---
# você pode escolher suas bordas; aqui uma grade razoável
rec_bins = [-1, 7, 30, 60, 90, 180, 365, 9999]
rec_labels = ['rec_0_7','rec_8_30','rec_31_60','rec_61_90','rec_91_180','rec_181_365','rec_365_plus']
df['recency_bin'] = pd.cut(df['recency_days'], bins=rec_bins, labels=rec_labels)
df = pd.get_dummies(df, columns=['recency_bin'], drop_first=False)

print('Feature Engineering concluído. Shape final:', df.shape)
display(df.head(3))


Base reduzida para: (776919, 12) linhas e 50000 clientes (top compradores)
Feature Engineering concluído. Shape final: (776919, 70)


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,...,monetary,avg_ticket,days_active,recency_bin_rec_0_7,recency_bin_rec_8_30,recency_bin_rec_31_60,recency_bin_rec_61_90,recency_bin_rec_91_180,recency_bin_rec_181_365,recency_bin_rec_365_plus
0,650efcd8ca4c7622a2232bba1a935704766ea4f7728415...,00015073181c1551292d45f1c06af91bcc4283442e1844...,2019-04-20,21:06:46,7688b6ef52555962d008fff894223582c484517cea7da4...,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c093...,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,...,1491.88,165.764444,1431,False,False,False,False,False,False,True
1,4b6ddadb0df37654e3993298e4325338c4789aadcfb7c3...,00015073181c1551292d45f1c06af91bcc4283442e1844...,2019-04-22,20:56:22,7688b6ef52555962d008fff894223582c484517cea7da4...,81b8a03f97e8787c53fe1a86bda042b6f0de9b0ec9c093...,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,...,1491.88,165.764444,1431,False,False,False,False,False,False,True
2,6a330dfa70428ffbba3ed17f1e66d9c053d9c74413445c...,00015073181c1551292d45f1c06af91bcc4283442e1844...,2019-11-18,11:14:43,7688b6ef52555962d008fff894223582c484517cea7da4...,274e64235a5fb9590afabdced2d45045b161d75e5748c2...,0,0,482d9673cfee5de391f97fde4d1c84f9f8d6f2cf0784fc...,1,...,1491.88,165.764444,1431,False,False,False,False,False,False,True


## 3. Elegibilidade e relatório de exclusões

In [ ]:

# Regras
cust_counts = df.groupby('fk_contact')[order_id_col].nunique().reset_index(name='n_orders')
single_customers = set(cust_counts[cust_counts['n_orders']==1]['fk_contact'])

last_purchase = df.groupby('fk_contact')['purchase_datetime'].max().reset_index()
last_purchase['days_since_last'] = (df['purchase_datetime'].max() - last_purchase['purchase_datetime']).dt.days
inactive_customers = set(last_purchase[last_purchase['days_since_last'] > 540]['fk_contact'])

ess_missing = set(df[df['purchase_datetime'].isna() | df['fk_contact'].isna() | df[order_id_col].isna()]['fk_contact'].dropna().unique())

exclude_customers = single_customers | inactive_customers | ess_missing
excluded_count = len(exclude_customers)
total_customers = df['fk_contact'].nunique()

print(f'Total customers: {total_customers} | Excluídos: {excluded_count} ({excluded_count/total_customers:.2%})')

# Salvar relatório
excl_df = pd.DataFrame({'fk_contact': list(exclude_customers)})
excl_df['reason'] = excl_df['fk_contact'].apply(lambda x: 'single_purchase' if x in single_customers else ('inactive_gt_540d' if x in inactive_customers else 'missing_ess'))
excl_path = os.path.join(OUTPUT_DIR,'exclusion_report_all_customers.csv')
excl_df.to_csv(excl_path, index=False)
if DRIVE_MOUNTED:
    excl_df.to_csv(os.path.join(DRIVE_OUT,'exclusion_report_all_customers.csv'), index=False)
print('Relatório salvo em:', excl_path)

# Base modelável
model_df = df[(df['purchase_number']>1) & (~df['fk_contact'].isin(exclude_customers))].copy()
print('model_df:', model_df.shape)


Total customers: 50000 | Excluídos: 14236 (28.47%)
Relatório salvo em: /mnt/data/exclusion_report_all_customers.csv
model_df: (557175, 70)


## 4. Parte 1 — Segmentação de clientes (RFM + KMeans)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# ===============================
# Construção do RFM
# ===============================
rfm = model_df.groupby('fk_contact').agg(
    recency_days=('recency_days','max'),
    frequency=('frequency','max'),
    monetary=('monetary','max'),
    avg_ticket=('avg_ticket','max'),
    days_active=('days_active','max')
).reset_index()

# Padronizar variáveis para cluster
X_seg = rfm[['recency_days','frequency','monetary','avg_ticket','days_active']].fillna(0).replace([np.inf,-np.inf],0)
X_seg_sc = StandardScaler().fit_transform(X_seg)

# KMeans com busca de k ótimo (2..8)
best_k, best_sil = 2, -1
for k in range(2,9):
    km = KMeans(n_clusters=k, random_state=42, n_init='auto').fit(X_seg_sc)
    sil = silhouette_score(X_seg_sc, km.labels_)
    if sil > best_sil:
        best_k, best_sil, best_model = k, sil, km

rfm['segment'] = best_model.labels_
print('Melhor k:', best_k, 'Silhouette:', round(best_sil,3))

# Nomes para os segmentos
segment_names = {
    0: "Usuários ocasionais (baixo gasto)",
    1: "Clientes fiéis de gasto médio",
    2: "Premium ocasionais (alto ticket, poucas compras)",
    3: "Super usuários (outliers / empresas)",
    4: "Clientes inativos / churnados",
    5: "Clientes VIP de alto valor"
}
rfm['segment_name'] = rfm['segment'].map(segment_names)

# ===============================
# Preview
# ===============================
display(rfm.head())
seg_profile = rfm.groupby(['segment','segment_name'])[['recency_days','frequency','monetary','avg_ticket','days_active']].mean().round(2)
display(seg_profile)

# ===============================
# Salvar segmentos COMPLETOS
# ===============================
seg_csv = os.path.join(OUTPUT_DIR,'customer_segments.csv')

# 🔑 Ajuste principal: salvar TODAS as colunas disponíveis
rfm.to_csv(seg_csv, index=False)

if DRIVE_MOUNTED:
    rfm.to_csv(os.path.join(DRIVE_OUT,'customer_segments.csv'), index=False)

print('Segments saved to:', seg_csv)


Melhor k: 6 Silhouette: 0.307


,fk_contact,recency_days,frequency,monetary,avg_ticket,days_active,segment,segment_name
0,00015073181c1551292d45f1c06af91bcc4283442e1844...,376,9,1491.88,165.764444,1431,4,Clientes inativos / churnados
1,0002b50f5c48ecd13312d5d52447551de23f63edab7667...,345,8,1348.16,168.520000,1995,4,Clientes inativos / churnados
2,0002fdedbce706df9a2602a5e1286a904e4582264572e1...,252,8,1650.90,206.362500,630,4,Clientes inativos / churnados
3,0004c81bb4ec2a4f2ed4abcb3647777ff84403c6940872...,412,14,2797.92,199.851429,1669,4,Clientes inativos / churnados
4,000b40069834aaf5757373f9aac1e92968ccbcfff3838c...,106,14,857.38,61.241429,868,0,Usuários ocasionais (baixo gasto)


,,recency_days,frequency,monetary,avg_ticket,days_active
segment,segment_name,,,,,
0,Usuários ocasionais (baixo gasto),80.88,15.50,1536.91,107.22,701.79
1,Clientes fiéis de gasto médio,120.70,20.09,2688.16,147.29,2188.19
2,"Premium ocasionais (alto ticket, poucas compras)",141.73,13.83,4597.87,340.54,1001.37
3,Super usuários (outliers / empresas),62.71,1037.14,287411.37,278.93,1975.29
4,Clientes inativos / churnados,387.94,13.10,1531.90,124.34,939.76
5,Clientes VIP de alto valor,115.81,217.89,49261.26,261.70,1601.99


Segments saved to: /mnt/data/customer_segments.csv


## 5. Parte 2 — Próxima compra (classificação 30 dias)

In [ ]:
from sklearn.model_selection import GroupKFold, cross_val_predict
from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.calibration import CalibratedClassifierCV
import optuna, lightgbm as lgb, joblib, os
import numpy as np

# ===============================
# Preparação dos dados
# ===============================
feature_candidates = [
    'cum_gmv_before','cum_tickets_before','avg_ticket_so_far',
    'days_since_prev','avg_days_between','interval_std','interval_cv',
    'unit_price','unit_price_rollmean',
    'unique_routes_before','same_route_as_last',
    'purchase_hour','purchase_weekday','purchase_month','is_weekend',
    'is_high_season','sin_month','cos_month','sin_wday','cos_wday','sin_hour','cos_hour',
    'route_count','route_avg_gmv','place_origin_departure_fe','place_destination_departure_fe',
    'fk_departure_ota_bus_company_fe','route_fe',
    'recency_days','frequency','monetary','avg_ticket','days_active',
    'is_holiday','is_holiday_window','days_to_holiday'
] + [c for c in df.columns if c.startswith('rec_')]  # rec_ dummies de recency

features = [f for f in feature_candidates if f in df.columns]
print(f"Total de features usadas: {len(features)}")

model_df = df.copy()  # se você já tem model_df, pode pular esta linha e trocar df->model_df abaixo
X = model_df[features].fillna(0)
y = model_df['label_30d'].astype(int)
groups = model_df['fk_contact']

gkf = GroupKFold(n_splits=5)

# Desbalanceamento (7)
pos_rate = float(y.mean()) if y.size > 0 else 0.5
scale_pos_weight = (1 - pos_rate) / (pos_rate + 1e-9) if pos_rate > 0 else 1.0
print(f"Positives ratio: {pos_rate:.4f} | scale_pos_weight: {scale_pos_weight:.2f}")

# ===============================
# Função de otimização (Optuna)
# ===============================
def objective(trial):
    params = {
        'objective':'binary','metric':'auc','verbosity':-1,'boosting_type':'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', 4, 16),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'n_estimators': 1000,
        'n_jobs':-1, 'random_state':42,
        'scale_pos_weight': scale_pos_weight
        # alternativa: 'is_unbalance': True
    }
    aucs = []
    for tr, va in gkf.split(X, y, groups):
        bst = lgb.LGBMClassifier(**params)
        bst.fit(
            X.iloc[tr], y.iloc[tr],
            eval_set=[(X.iloc[va], y.iloc[va])],
            eval_metric='auc',
            callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
        )
        proba = bst.predict_proba(X.iloc[va])[:,1]
        aucs.append(roc_auc_score(y.iloc[va], proba))
    return float(np.mean(aucs))

# ===============================
# Executar estudo do Optuna
# ===============================
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, show_progress_bar=False)

best_params = study.best_params
best_params.update({
    'objective':'binary','metric':'auc','n_jobs':-1,'random_state':42,'n_estimators':1000,
    'scale_pos_weight': scale_pos_weight
})
print('Melhores parâmetros:', best_params)

# ===============================
# Modelo final LightGBM
# ===============================
final_lgb = lgb.LGBMClassifier(**best_params)
final_lgb.fit(
    X, y,
    eval_set=[(X, y)],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

# ===============================
# Validação cruzada LightGBM
# ===============================
cv_probs_lgb = cross_val_predict(final_lgb, X, y, groups=groups, cv=gkf, method='predict_proba')[:,1]
auc_lgb = roc_auc_score(y, cv_probs_lgb)
print('CV ROC AUC (LightGBM):', round(auc_lgb, 6))

# ===============================
# Ensemble opcional (6) com XGBoost / CatBoost se disponíveis
# ===============================
cv_probs_ens_list = [cv_probs_lgb]
try:
    import xgboost as xgb
    xgb_params = dict(
        n_estimators=500, learning_rate=best_params.get('learning_rate', 0.05),
        max_depth=best_params.get('max_depth', 8),
        subsample=best_params.get('subsample', 0.8),
        colsample_bytree=best_params.get('colsample_bytree', 0.8),
        reg_alpha=best_params.get('reg_alpha', 0.0),
        reg_lambda=best_params.get('reg_lambda', 1.0),
        random_state=42, n_jobs=-1,
        eval_metric='auc',
        scale_pos_weight=scale_pos_weight
    )
    xgb_model = xgb.XGBClassifier(**xgb_params)
    cv_probs_xgb = cross_val_predict(xgb_model, X, y, groups=groups, cv=gkf, method='predict_proba')[:,1]
    print('CV ROC AUC (XGBoost):', round(roc_auc_score(y, cv_probs_xgb), 6))
    cv_probs_ens_list.append(cv_probs_xgb)
except Exception as e:
    print('XGBoost indisponível/erro, seguindo sem ele ->', e)

try:
    from catboost import CatBoostClassifier
    cat_params = dict(
        iterations=500, learning_rate=best_params.get('learning_rate', 0.05),
        depth=min(max(best_params.get('max_depth', 8), 4), 10),
        loss_function='Logloss', eval_metric='AUC',
        random_seed=42, verbose=False, class_weights=[1.0, float(scale_pos_weight)]
    )
    cat = CatBoostClassifier(**cat_params)
    # CatBoost não suporta groups diretamente em cross_val_predict; vamos usar KFold com split manual
    cv_probs_cat = np.zeros(len(X))
    for tr_idx, va_idx in gkf.split(X, y, groups):
        cat.fit(X.iloc[tr_idx], y.iloc[tr_idx], eval_set=(X.iloc[va_idx], y.iloc[va_idx]))
        cv_probs_cat[va_idx] = cat.predict_proba(X.iloc[va_idx])[:,1]
    print('CV ROC AUC (CatBoost):', round(roc_auc_score(y, cv_probs_cat), 6))
    cv_probs_ens_list.append(cv_probs_cat)
except Exception as e:
    print('CatBoost indisponível/erro, seguindo sem ele ->', e)

# Blending simples (nesse caso, só o LGBM)
cv_probs_blend = np.mean(np.vstack(cv_probs_ens_list), axis=0)
auc_blend = roc_auc_score(y, cv_probs_blend)
print('CV ROC AUC (Blend disponível):', round(auc_blend, 6))

# ===============================
# Calibração (com groups) do melhor fluxo
# ===============================
# Se o blend for melhor, calibramos sobre as probabilidades do LGBM final
# (Obs.: para calibrar o blend corretamente, teríamos que calibrar cada modelo
# e depois combinar; aqui mantemos a calibração no final_lgb para simplicidade/velocidade.)
cal = CalibratedClassifierCV(final_lgb, method='isotonic', cv=5)
cal.fit(X, y)


cal_probs = cal.predict_proba(X)[:,1]
prec, rec, thr = precision_recall_curve(y, cal_probs)
f1 = 2*(prec*rec)/(prec+rec+1e-9)
best_idx = int(np.nanargmax(f1))
best_thr = thr[best_idx] if best_idx < len(thr) else 0.5
print('Threshold ótimo (F1):', best_thr)

# ===============================
# Salvar modelo
# ===============================
OUTPUT_DIR = OUTPUT_DIR if 'OUTPUT_DIR' in globals() else "."
model_art = {
    'model': final_lgb,
    'calibrator': cal,
    'features': features,
    'threshold': best_thr,
    'auc_lgb_cv': float(auc_lgb),
    'auc_blend_cv': float(auc_blend)
}
model_path = os.path.join(OUTPUT_DIR,'model_clickbus_v8.joblib')
joblib.dump(model_art, model_path)
if 'DRIVE_MOUNTED' in globals() and DRIVE_MOUNTED:
    DRIVE_OUT = DRIVE_OUT if 'DRIVE_OUT' in globals() else OUTPUT_DIR
    joblib.dump(model_art, os.path.join(DRIVE_OUT,'model_clickbus_v8.joblib'))
print('Modelo salvo em:', model_path)
metrics = {
    "auc_lgb_cv": round(auc_lgb, 6),
    "auc_blend_cv": round(auc_blend, 6),
    "best_threshold": round(best_thr, 4),
    "pos_rate": round(pos_rate, 4)
}

metrics_df = pd.DataFrame([metrics])
metrics_csv = os.path.join(OUTPUT_DIR, "training_metrics.csv")
metrics_df.to_csv(metrics_csv, index=False)
if DRIVE_MOUNTED:
    metrics_df.to_csv(os.path.join(DRIVE_OUT, "training_metrics.csv"), index=False)

print("Métricas de treino salvas em:", metrics_csv)
print("Métricas de treino:")
display(metrics_df)

Total de features usadas: 36


[I 2025-09-21 14:54:05,859] A new study created in memory with name: no-name-6ce372e9-3f6a-47a1-8e72-9dde9dd7e204


Positives ratio: 0.6002 | scale_pos_weight: 0.67


[I 2025-09-21 15:11:57,017] Trial 0 finished with value: 0.8119409477911093 and parameters: {'learning_rate': 0.013835032132595093, 'num_leaves': 60, 'max_depth': 11, 'min_child_samples': 75, 'subsample': 0.8032554740389626, 'colsample_bytree': 0.7361598403755125, 'reg_alpha': 0.0092368652501666, 'reg_lambda': 3.436601404798516e-06}. Best is trial 0 with value: 0.8119409477911093.
[I 2025-09-21 15:28:31,829] Trial 1 finished with value: 0.8200849405428979 and parameters: {'learning_rate': 0.015371382110590272, 'num_leaves': 210, 'max_depth': 15, 'min_child_samples': 29, 'subsample': 0.8071440032710785, 'colsample_bytree': 0.4238548629184896, 'reg_alpha': 0.2555819174586763, 'reg_lambda': 0.0017386553777781534}. Best is trial 1 with value: 0.8200849405428979.
[I 2025-09-21 15:46:59,094] Trial 2 finished with value: 0.8167011368117556 and parameters: {'learning_rate': 0.017291541507046013, 'num_leaves': 225, 'max_depth': 8, 'min_child_samples': 71, 'subsample': 0.5001553922982747, 'colsa

Melhores parâmetros: {'learning_rate': 0.0648295517707632, 'num_leaves': 183, 'max_depth': 13, 'min_child_samples': 48, 'subsample': 0.5210574266092416, 'colsample_bytree': 0.9775751628993485, 'reg_alpha': 3.8379506409241875e-06, 'reg_lambda': 0.15092327182408324, 'objective': 'binary', 'metric': 'auc', 'n_jobs': -1, 'random_state': 42, 'n_estimators': 1000, 'scale_pos_weight': 0.6660640750611645}
CV ROC AUC (LightGBM): 0.824958
CV ROC AUC (XGBoost): 0.810885
CV ROC AUC (CatBoost): 0.815735
CV ROC AUC (Blend disponível): 0.823155
Threshold ótimo (F1): 0.46394755229176204
Modelo salvo em: /mnt/data/model_clickbus_v8.joblib
Métricas de treino salvas em: /mnt/data/training_metrics.csv
Métricas de treino:


,auc_lgb_cv,auc_blend_cv,best_threshold,pos_rate
0,0.824958,0.823155,0.4639,0.6002


In [ ]:
'''import joblib
import os

# Caminho do modelo e métricas já salvos
model_path = "/mnt/data/model_clickbus_v8.joblib"
metrics_path = "/mnt/data/training_metrics.csv"

# Carregar modelo calibrado
model_art = joblib.load(model_path)

# Acessar partes do artefato
final_model = model_art['model']          # LightGBM treinado
calibrator = model_art['calibrator']      # Calibrador isotônico
features = model_art['features']          # Features usadas
threshold = model_art['threshold']        # Melhor threshold encontrado

print("Modelo carregado com sucesso ✅")
print("AUC CV LightGBM:", model_art['auc_lgb_cv'])
print("AUC CV Blend:", model_art['auc_blend_cv'])
print("Threshold ótimo:", threshold)

# Carregar métricas
import pandas as pd
metrics_df = pd.read_csv(metrics_path)
print("\nMétricas salvas:")
display(metrics_df)'''


Modelo carregado com sucesso ✅
AUC CV LightGBM: 0.8249582720724917
AUC CV Blend: 0.8231548589662502
Threshold ótimo: 0.46394755229176204

Métricas salvas:


,auc_lgb_cv,auc_blend_cv,best_threshold,pos_rate
0,0.824958,0.823155,0.4639,0.6002


## 6. Regressão — Dias até a próxima compra

In [ ]:
import lightgbm as lgb

# ===============================
# Treino do regressor
# ===============================
reg_df = model_df.dropna(subset=['days_to_next']).copy()
Yr = reg_df['days_to_next'].clip(lower=0, upper=180).astype(float)
Xr = reg_df[features].fillna(0)

reg = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1
)
reg.fit(Xr, Yr)

# ===============================
# Previsão da próxima compra (última compra de cada cliente)
# ===============================
last_snap = model_df.sort_values("purchase_datetime").groupby("fk_contact").tail(1).reset_index(drop=True)

X_last = last_snap[features].fillna(0)
y_pred_days = reg.predict(X_last).clip(0, 180)

last_snap['last_purchase_date'] = last_snap['purchase_datetime']
last_snap['pred_days_to_next'] = y_pred_days.astype(int)
last_snap['pred_next_purchase_date'] = last_snap['last_purchase_date'] + pd.to_timedelta(last_snap['pred_days_to_next'], unit="D")

# ===============================
# Exportar TODAS as colunas + previsões
# ===============================
csv_next = os.path.join(OUTPUT_DIR,'pred_next_purchase_dates.csv')
last_snap.to_csv(csv_next, index=False)

if DRIVE_MOUNTED:
    last_snap.to_csv(os.path.join(DRIVE_OUT,'pred_next_purchase_dates.csv'), index=False)

print("Arquivo com previsões de próxima compra salvo em:", csv_next)
display(last_snap.head(10))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.100173 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5557
[LightGBM] [Info] Number of data points in the train set: 521411, number of used features: 36
[LightGBM] [Info] Start training from score 40.903506
Arquivo com previsões de próxima compra salvo em: /mnt/data/pred_next_purchase_dates.csv


,nk_ota_localizer_id,fk_contact,date_purchase,time_purchase,place_origin_departure,place_destination_departure,place_origin_return,place_destination_return,fk_departure_ota_bus_company,fk_return_ota_bus_company,...,recency_bin_rec_0_7,recency_bin_rec_8_30,recency_bin_rec_31_60,recency_bin_rec_61_90,recency_bin_rec_91_180,recency_bin_rec_181_365,recency_bin_rec_365_plus,last_purchase_date,pred_days_to_next,pred_next_purchase_date
0,dda45499159b86dbd45e6313fe291ac104debf9635b3f5...,faf7c425f68ce851d1057189ba5abf9b36945e8bb76ab6...,2022-10-09,06:28:15,0f33e04ac3b3762d50f9530393b85019c1d935777def12...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,0,0,dbae772db29058a88f9bd830e957c695347c41b6162a7e...,1,...,False,False,False,False,False,False,True,2022-10-09 06:28:15,54,2022-12-02 06:28:15
1,0c8f96996d28505db78d455e693d8eaf28f38d52f40318...,400cc5d48ac37490a2e1c084e4ec6ff472218d889991a1...,2022-10-09,08:38:52,5904c93073387f898f4f339094d4bad7043b6440519deb...,285b71922aaa01d870483d3b4f59e4a61c2057e1476854...,0,0,9bdb2af6799204a299c603994b8e400e4b1fd625efdb74...,1,...,False,False,False,False,False,False,True,2022-10-09 08:38:52,65,2022-12-13 08:38:52
2,543a8f81f6839de2b05e0b53efd81b464201db0f79272a...,e4cddef84a09b2929802adc4f3009a4917db371d6f856d...,2022-10-09,09:41:30,131b0c35e2d7edef9dd63f48eff39341ef0a5f770538aa...,aaf5060b9517ba4f550ee34a7f3ed7b05b6e5100a523d3...,0,0,7902699be42c8a8e46fbbb4501726517e86b22c56a189f...,1,...,False,False,False,False,False,False,True,2022-10-09 09:41:30,19,2022-10-28 09:41:30
3,d2632a257a1932459585f813214d7124dbcf259362d2bc...,73603f869e53a5cbe0295bf089db47f4a789da67a7fb1e...,2022-10-09,10:11:01,20ca98162ba780883712eb701c84e4c06f73aba78e9039...,f4dd301311d96b70a2ee62a6bccfe21bb0d94a89ca2805...,0,0,2747b7c718564ba5f066f0523b03e17f6a496b06851333...,1,...,False,False,False,False,False,False,True,2022-10-09 10:11:01,62,2022-12-10 10:11:01
4,87ad82b83f3049091f55efe927d1101731ca176bb6c313...,89889ae45dd6fcaefac94ba12b0a0903e9b4edb9ce848a...,2022-10-09,10:54:28,fed88b40aba63cac05eadd5db0088c036005ec235c7be6...,2b9449f314bf93145f8122906d8dc56c4ca1f116e6db7a...,0,0,2747b7c718564ba5f066f0523b03e17f6a496b06851333...,1,...,False,False,False,False,False,False,True,2022-10-09 10:54:28,58,2022-12-06 10:54:28
5,9fa3b7919b3e8280e28c054a77948001a09cbe2b90fd93...,a6e0b88defdd5a3902efa185f54f8bc77ddc31f0e79c1f...,2022-10-09,11:39:33,fbb2a73b0bacf3953186a92029e3e9b130373a9ff14494...,3caffc05c7f0e2906e518b7c96b1a3bab9ae1ed894b83d...,0,0,02d20bbd7e394ad5999a4cebabac9619732c343a4cac99...,1,...,False,False,False,False,False,False,True,2022-10-09 11:39:33,49,2022-11-27 11:39:33
6,acd3d57a364a31d2a440b7fda4dcf344b794db3fe43f31...,c51b8079e5db5656856ce53cbdb6a4419bd01b4e1ed06c...,2022-10-09,12:24:36,ffd560d182369b08a8b3ed35cfa5ee3cc50b5b5f093ece...,e0e3fcdfb0803ca538910c12042cb6232acb9e3c6db890...,0,0,ec2e990b934dde55cb87300629cedfc21b15cd28bbcf77...,1,...,False,False,False,False,False,False,True,2022-10-09 12:24:36,51,2022-11-29 12:24:36
7,41661ccb812ded251939f55a5abee97734f2f0a2b04712...,d12c380ec3b04b10a1699a123110c52309e7fd418ccb13...,2022-10-09,12:53:43,f44b7809595e5ebe3bb4d65e0cdd1ebbb41b29f5ab8fd7...,fc72c98a6c2916c1bbf9f39fce094f5785bb6f1d656971...,0,0,3ada92f28b4ceda38562ebf047c6ff05400d4c572352a1...,1,...,False,False,False,False,False,False,True,2022-10-09 12:53:43,49,2022-11-27 12:53:43
8,01174db1ba84f3647c90206315bf77ed397f19fe0607c5...,edc3fda4884a8b0144ceaf4676063b3b8c43de1fc17138...,2022-10-09,13:07:21,670671cd97404156226e507973f2ab8330d3022ca96e0c...,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1...,1,...,False,False,False,False,False,False,True,2022-10-09 13:07:21,65,2022-12-13 13:07:21
9,831e720078bbbc441d9bcf6b01a404eee4cd27f86f7bd0...,3ed82638617ac57086a1addc5e51f932f935e88753baf9...,2022-10-09,13:16:02,6b51d431df5d7f141cbececcf79edf3dd861c3b4069f0b...,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,0,0,35135aaa6cc23891b40cb3f378c53a17a1127210ce60e1.

## 7. Próximo trecho — Classificação multiclasses (baseline)

In [ ]:
import numpy as np
import pandas as pd
import time, os, joblib
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import early_stopping

# ================================
# Configurações
# ================================
OUTPUT_DIR = "./output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

USE_SAMPLE = False
SAMPLE_SIZE = 20000
BATCH_SIZE = 5000

# ================================
# Preparação dos dados
# ================================
X_route = df[features].fillna(0)
y_raw = df["route_fe"].astype(str)

le = LabelEncoder()
y_route = le.fit_transform(y_raw)

X_train, X_val, y_train, y_val = train_test_split(
    X_route, y_route, test_size=0.2, random_state=42, stratify=y_route
)

# ================================
# Treino do modelo
# ================================
route_clf = lgb.LGBMClassifier(
    objective="multiclass",
    num_class=len(np.unique(y_route)),
    learning_rate=0.1,
    n_estimators=500,
    random_state=42,
    n_jobs=-1
)

route_clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="multi_logloss",
    callbacks=[early_stopping(stopping_rounds=30, verbose=False)]
)

# Salvar artefatos
joblib.dump(route_clf, os.path.join(OUTPUT_DIR, "route_model.joblib"))
joblib.dump(le, os.path.join(OUTPUT_DIR, "route_labelencoder.joblib"))
print("✅ Modelo e LabelEncoder salvos")

# ================================
# Funções auxiliares
# ================================
def predict_in_batches(model, X, batch_size=BATCH_SIZE):
    """Predição em lotes com ETA."""
    n = len(X)
    n_batches = int(np.ceil(n / batch_size))
    all_probs = []

    start_time = time.time()
    for i in range(n_batches):
        batch = X.iloc[i*batch_size:(i+1)*batch_size]
        all_probs.append(model.predict_proba(batch))

        elapsed = time.time() - start_time
        avg = elapsed / (i+1)
        eta = avg * (n_batches - (i+1))
        print(f"[{i+1}/{n_batches}] {elapsed/60:.1f} min elapsed | ETA {eta/60:.1f} min")

    return np.vstack(all_probs)

def topk_accuracy(y_true, probs, k=3):
    """Acurácia Top-K vetorizada."""
    topk = np.argsort(probs, axis=1)[:, -k:]
    return np.mean([yt in tk for yt, tk in zip(y_true, topk)])

# ================================
# Avaliação
# ================================
if USE_SAMPLE:
    idx = np.random.choice(len(X_route), size=min(SAMPLE_SIZE, len(X_route)), replace=False)
    X_eval, y_eval = X_route.iloc[idx], y_route[idx]
else:
    X_eval, y_eval = X_route, y_route

probs = predict_in_batches(route_clf, X_eval, batch_size=BATCH_SIZE)

metrics = {
    "Top1": round(topk_accuracy(y_eval, probs, 1), 4),
    "Top3": round(topk_accuracy(y_eval, probs, 3), 4),
    "Top5": round(topk_accuracy(y_eval, probs, 5), 4)
}
print("✅ Métricas:", metrics)

# Salvar métricas
pd.DataFrame([metrics]).to_csv(os.path.join(OUTPUT_DIR, "route_metrics.csv"), index=False)

# ================================
# Exportar recomendações Top-3
# ================================
pred_top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
pred_labels = le.inverse_transform(pred_top3.ravel()).reshape(pred_top3.shape)

# Recupera o mesmo subset no df original (se usou amostra)
df_eval = df.iloc[X_eval.index].copy()

# Junta com as predições
route_preds = df_eval.copy()
route_preds["y_true"] = le.inverse_transform(y_eval)
route_preds["pred_top1"] = pred_labels[:, 0]
route_preds["pred_top2"] = pred_labels[:, 1]
route_preds["pred_top3"] = pred_labels[:, 2]

# Exporta
route_csv = os.path.join(OUTPUT_DIR, "route_recommendations.csv")
route_preds.to_csv(route_csv, index=False)
print("✅ Arquivo com TODAS as colunas + predições salvo em:", route_csv)

A saída de streaming foi truncada nas últimas 5000 linhas.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

## 8. Explainability — SHAP

In [ ]:

import shap
explainer = shap.TreeExplainer(final_lgb)
sample_idx = np.random.choice(X.index, size=min(4000, len(X)), replace=False)
X_sample = X.loc[sample_idx]
sv = explainer.shap_values(X_sample)

# Summary
import matplotlib.pyplot as plt
shap.summary_plot(sv, X_sample, show=False)
plt.tight_layout()
shap_png = os.path.join(OUTPUT_DIR,'shap_summary_v5.png')
plt.savefig(shap_png, dpi=200)
if DRIVE_MOUNTED:
    plt.savefig(os.path.join(DRIVE_OUT,'shap_summary_v5.png'), dpi=200)
print('SHAP summary salvo:', shap_png)

# Importância média absoluta dos SHAP values
shap_importance = pd.DataFrame({
    "feature": X_sample.columns,
    "importance": np.abs(sv).mean(axis=0)
}).sort_values("importance", ascending=False)

shap_csv = os.path.join(OUTPUT_DIR, "shap_importance.csv")
shap_importance.to_csv(shap_csv, index=False)
if DRIVE_MOUNTED:
    shap_importance.to_csv(os.path.join(DRIVE_OUT, "shap_importance.csv"), index=False)

print("Importâncias SHAP salvas em:", shap_csv)



## 9. Predições finais (<= 50k), artefatos e export

In [ ]:
# Snapshot: última compra por cliente elegível
snap = model_df.sort_values('purchase_datetime').groupby('fk_contact').tail(1).reset_index(drop=True)
Xs = snap[features].fillna(0)
snap['prob_next_30d'] = cal.predict_proba(Xs)[:,1]
snap['pred_next_30d'] = (snap['prob_next_30d'] >= best_thr).astype(int)

# Limitar a 50k
snap_50k = snap.sort_values('prob_next_30d', ascending=False).head(50000).copy()

# 🔹 Salvar todas as colunas disponíveis + probabilidade + previsão
pred_csv = os.path.join(OUTPUT_DIR, 'predicoes_clickbus_snapshot_30d_v5.csv')
snap_50k.to_csv(pred_csv, index=False)

if DRIVE_MOUNTED:
    snap_50k.to_csv(os.path.join(DRIVE_OUT, 'predicoes_clickbus_snapshot_30d_v5.csv'), index=False)

print('Predições salvas em:', pred_csv)

# Versão "fantasiosa"
demo = snap_50k.copy()
first = ['Lucas','Mariana','Pedro','Ana','Carlos','Fernanda','Rafael','Beatriz','Guilherme','Laura']
last  = ['Silva','Souza','Oliveira','Pereira','Costa','Almeida','Gomes','Ribeiro','Martins']
cities = ['São Paulo','Rio de Janeiro','Belo Horizonte','Porto Alegre','Salvador','Fortaleza','Curitiba','Recife']
companies = ['BusFast','ViaRápida','Rodaria','ExpressoSul','AutoBus Pro']

rng = np.random.default_rng(42)
demo = demo.sample(n=min(500, len(demo)), random_state=42).reset_index(drop=True)
demo['name'] = [rng.choice(first) + ' ' + rng.choice(last) + f' #{i%1000}' for i in range(len(demo))]
demo['city'] = [rng.choice(cities) for _ in range(len(demo))]
demo['company'] = [rng.choice(companies) for _ in range(len(demo))]

# 🔹 Salvar demo com todas as colunas + extras
demo_csv = os.path.join(OUTPUT_DIR, 'predicoes_clickbus_snapshot_30d_v5_demo.csv')
demo.to_csv(demo_csv, index=False)

if DRIVE_MOUNTED:
    demo.to_csv(os.path.join(DRIVE_OUT, 'predicoes_clickbus_snapshot_30d_v5_demo.csv'), index=False)

print('Demo CSV salvo em:', demo_csv)


## 10. App Streamlit (apresentação rápida)

In [ ]:
'''!pip install streamlit pyngrok -q

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Criar o arquivo app.py com o código do dashboard
with open("app.py", "w", encoding="utf-8") as f:
    f.write("""
# -*- coding: utf-8 -*-
import os
import io
import time
import numpy as np
import pandas as pd
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go

# =========================
# CONFIG & ESTILO
# =========================
st.set_page_config(
    page_title="ClickBus • Growth AI",
    page_icon="🚌",
    layout="wide",
    initial_sidebar_state="expanded"
)

PRIMARY = "#ff3d00"
DARK    = "#111827"
ACCENT  = "#00c2ff"
OK      = "#22c55e"
WARN    = "#f59e0b"

st.markdown(f\"\"\"
<style>
    .stApp {{
        background: radial-gradient(1200px 800px at 10% 10%, #0b1220 0%, #0b1220 40%, #0d1324 70%, #0e1529 100%);
        color: #e5e7eb;
        font-family: 'Inter', system-ui, -apple-system, Segoe UI, Roboto, Helvetica, Arial, sans-serif;
    }}
    h1, h2, h3 {{
        color: {ACCENT};
    }}
    .metric-card {{
        background: linear-gradient(180deg, rgba(255,255,255,0.06), rgba(255,255,255,0.02));
        border-radius: 16px;
        padding: 16px 18px;
    }}
    .big-number {{
        font-size: 34px; font-weight: 800; color: white;
    }}
</style>
\"\"\", unsafe_allow_html=True)

# =========================
# UTILS
# =========================
@st.cache_data(show_spinner=False)
def try_load_csv(paths):
    for p in paths:
        if isinstance(p, (tuple, list)) and len(p) == 2:
            base, name = p
            full = os.path.join(base, name)
        else:
            full = p
        if os.path.exists(full):
            try:
                return pd.read_csv(full), full
            except Exception:
                try:
                    return pd.read_csv(full, sep=";"), full
                except Exception:
                    pass
    return None, None

def kpi_card(title, value, color=ACCENT):
    st.markdown(f"<div class='metric-card'><div class='big-number' style='color:{color}'>{value}</div><p>{title}</p></div>", unsafe_allow_html=True)

# =========================
# DETECTAR AMBIENTE
# =========================
default_dirs = [
    "/content/drive/MyDrive/clickbuss",      # Colab/Drive
    "C:\\\\Users\\\\isaac\\\\Documents\\\\clickbuss",  # Windows
]

base_dir = default_dirs[0] if os.path.exists(default_dirs[0]) else default_dirs[1]

# =========================
# CARREGAMENTO DE ARQUIVOS
# =========================
pred_df, pred_path = try_load_csv([(base_dir, "predicoes_clickbus_snapshot_30d_v5.csv")])
seg_df, seg_path   = try_load_csv([(base_dir, "customer_segments.csv")])
rec_df, rec_path   = try_load_csv([(base_dir, "route_recommendations.csv")])

if pred_df is None:
    st.error("Arquivo de predições não encontrado!")
else:
    st.success(f"Predições carregadas de {pred_path}")

if seg_df is None:
    st.warning("Arquivo de segmentos não encontrado. Usando demo.")
    seg_df = pred_df.copy()
    seg_df["segment"] = np.random.choice(["VIP","Leal","Churn","Regular"], size=len(seg_df))

if rec_df is None:
    st.warning("Arquivo de rotas não encontrado. Usando demo.")
    rec_df = pred_df[["fk_contact"]].copy()
    rec_df["route@top1"] = np.random.choice(["SPO->RIO","RIO->SPO"], size=len(rec_df))
    rec_df["route@top2"] = np.random.choice(["SSA->REC","REC->SSA"], size=len(rec_df))
    rec_df["route@top3"] = np.random.choice(["BHZ->SPO","SPO->BHZ"], size=len(rec_df))

merged = pred_df.merge(seg_df, on="fk_contact", how="left")

# =========================
# NAVEGAÇÃO
# =========================
page = st.sidebar.radio(
    "Navegar",
    ["📊 Próxima Compra", "👥 Segmentos", "🛣️ Rotas"],
    index=0
)

# =========================
# PÁGINAS
# =========================
if page == "📊 Próxima Compra":
    st.header("📊 Probabilidade de próxima compra")
    st.dataframe(pred_df.head(20))

elif page == "👥 Segmentos":
    st.header("👥 Segmentos de clientes")
    st.dataframe(seg_df.head(20))

elif page == "🛣️ Rotas":
    st.header("🛣️ Recomendações de rotas")
    st.dataframe(rec_df.head(20))
""")

# Rodar o Streamlit via ngrok
from pyngrok import ngrok
!streamlit run app.py &>/dev/null&
url = ngrok.connect(8501)
print("✅ Acesse o Streamlit aqui:", url)'''


## 11. Slides (PowerPoint) — resumo executivo

In [ ]:

from pptx import Presentation
from pptx.util import Inches
prs = Presentation()
title_slide = prs.slides.add_slide(prs.slide_layouts[0])
title_slide.shapes.title.text = "ClickBus — Entrega Completa (v5)"
title_slide.placeholders[1].text = "Segmentação • Próxima compra • Próximo trecho • Explainability • Entregáveis"

for t in [
    "Objetivos & Escopo",
    "Dados & Dicionário",
    "Feature Engineering",
    "Modelagem (Classificação, Regressão, Survival)",
    "Próximo Trecho (Multiclasse)",
    "Segmentação (RFM + Perfis)",
    "Métricas (AUC/F1) & SHAP",
    "Entregáveis & Recomendações"
]:
    s = prs.slides.add_slide(prs.slide_layouts[1])
    s.shapes.title.text = t
    s.placeholders[1].text = "Pontos-chave / gráficos / tabelas / decisões."
ppt_path = os.path.join(OUTPUT_DIR,'ClickBus_v5_resumo.pptx')
prs.save(ppt_path)
if DRIVE_MOUNTED:
    prs.save(os.path.join(DRIVE_OUT,'ClickBus_v5_resumo.pptx'))
print('Slides salvos em:', ppt_path)


In [ ]:
!cp /mnt/data/predicoes_clickbus_snapshot_30d_v5.csv /content/
!cp /mnt/data/predicoes_clickbus_snapshot_30d_v5_demo.csv /content/
!cp /mnt/data/streamlit_app_v5.py /content/



### Artefatos gerados (após executar):
- `exclusion_report_all_customers.csv`
- `model_clickbus_v5.joblib`
- `predicoes_clickbus_snapshot_30d_v5.csv` (≤50k)
- `predicoes_clickbus_snapshot_30d_v5_demo.csv`
- `shap_summary_v5.png`
- `streamlit_app_v5.py`
- `ClickBus_v5_resumo.pptx`
Todos salvos em **/content/drive/MyDrive/clickbuss** (quando Drive montado) e **/mnt/data** (para baixar aqui).
